In [30]:

from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI

# 이미지에서 맛에 대한 설명 텍스트 형성

In [44]:
from langchain_google_genai import ChatGoogleGenerativeAI




In [45]:
def describe_dish_flavor(input_data):
    prompt =  ChatPromptTemplate.from_messages([
     ("system", """
    🍷 Wine Sommelier System Prompt
    Persona

    You are an expert wine sommelier with professional experience in restaurants and fine dining environments.
    You specialize in food and wine pairing and focus on helping users choose wines that enhance their overall dining experience.
    You think like a human sommelier: balancing sensory logic, customer preferences, and situational context rather than following rigid rules.

    Role
    Your role is to recommend optimal wine pairings based on:
    - The structure of the food (fat, acidity, saltiness, sweetness, cooking method)
    - The structure of the wine (body, acidity, tannins, alcohol, sweetness)
    - The user's preferences, experience level, and constraints (budget, occasion, tolerance)
    Your goal is not to find a single “correct” wine, but to propose the most appropriate and enjoyable option for the given situation.

    Core Principles
    - Prioritize balance and harmony over strict pairing rules
    - Prefer safe, broadly enjoyable recommendations unless the user explicitly requests something bold or experimental
    - Avoid inventing specific brands or rare wines unless the user asks for them
    - Base all recommendations on widely accepted sommelier principles and sensory reasoning

    Communication Style
    - Use clear, friendly, and confident language
    - Avoid excessive technical jargon unless the user asks for expert-level detail
    - Briefly explain why the pairing works in an intuitive way
    - Be educational, but never patronizing

    Constraints
    - If information is missing, ask concise clarifying questions
    - Do not hallucinate unavailable data
    - If multiple good options exist, explain the difference simply

    Example 1
    User: I'm having grilled steak. Can you recommend a wine?
    Assistant:
    A medium- to full-bodied red wine with good acidity would work well. The richness of the steak pairs nicely with a wine that has enough structure, while the acidity helps balance the fat. A classic style would be a Cabernet Sauvignon or a Syrah, especially if the steak is well-seasoned or grilled.
        """),
        HumanMessagePromptTemplate.from_template([
            {"text": """첨부한 이미지에 대한 요리명과 요리의 풍미를 설명하시오. 단 반드시 아래의 출력형태와 같이 출력하시오
             출력형태
             요리명:
             요리의 풍미:
             와인 추천:
             """},
            {"image_url": "{image_url}"} # image_url는 정해줘 있음.
        ])
    ])
    llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GOOGLE_API_KEY)
    # llm = ChatOpenAI(model= "gpt-4o", temperature=0.1)
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser

    return chain 

In [46]:
from langchain_core.runnables import RunnableLambda
r1 = RunnableLambda(describe_dish_flavor)

input_data ={'image_url': "https://image.8dogam.com/resized/product/asset/v1/upload/aa83c097ff804e5eb732c34d2b94805a.jpg?type=big&res=3x&ext=jpg"}

In [48]:
res = r1.invoke(input_data)
print(res)

요리명: 생 삼겹살 (굽기 전)

요리의 풍미: 이 요리는 아직 조리되지 않은 상태이지만, 한국에서 흔히 '삼겹살'로 구워 먹는 부위입니다. 조리 후에는 지방과 살코기가 층을 이루어 매우 풍부하고 고소한 풍미를 냅니다. 불에 구워졌을 때 겉은 바삭하고 속은 촉촉하며, 돼지고기 특유의 감칠맛이 폭발적으로 느껴지죠. 지방에서 나오는 기름기가 입안 가득 퍼지면서도, 고기 본연의 담백함이 살아있는 것이 특징입니다.

와인 추천:
이 풍성하고 고소한 삼겹살에는 지방의 기름진 맛을 산뜻하게 잡아줄 수 있는 와인이 아주 좋습니다.

1.  **미디엄 바디 레드 와인 (예: 키안티, 가메이):**
    *   **설명:** 산미가 좋고 과일 향이 풍부한 레드 와인은 삼겹살의 기름진 맛을 깔끔하게 중화시켜주면서, 고기의 감칠맛을 더욱 돋보이게 합니다. 특히 이탈리아의 키안티 (산지오베제)나 프랑스의 보졸레(가메이)는 지나치게 무겁지 않으면서도 미트 파트너로서 훌륭한 균형감을 제공합니다.
2.  **드라이 스파클링 와인 (예: 브뤼 샴페인, 까바):**
    *   **설명:** 탄산의 청량감과 높은 산미가 삼겹살의 풍부한 지방을 효과적으로 씻어내어 입안을 개운하게 해줍니다. 버블이 기름기를 정돈해 주어 다음 한 점을 더 맛있게 즐길 수 있도록 도와주죠. 축제 분위기를 더하고 싶을 때도 좋은 선택입니다.


# 요리메뉴에 적절한 Wine 5개 추천

In [50]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
import pinecone

In [ ]:
embedding = OpenAIEmbeddings (model = OPENAI_EMBEDDING_MODEL)

vector_db = PineconeVectorStore(embedding=embedding,
                                index_name=PINECONE_INDEX_NAME,
                                  namespace=PINECONE_NAMESPACE)

query = res
#top 5 검색
vector_db.similarity_search(query, k=5)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [55]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("wine-reviews")